# Assistant-03<br/>Function calling - Trading Assistant

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as helper
import yfinance as yf

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Retreive the latest stock price using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

### Prepare the tool for function calling

In [3]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

### Create the assistant

In [4]:
helper.clear_shelves()

## Create an assistant
stock_assistant = helper.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list, 
                                                  model=helper.gpt_deployment_name)

Added assistant:  asst_AJ5ghb5I9q7ZCVnk9WQxxuBJ 1


### Handle Processing functions

In [5]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "get_stock_price":
            output = get_stock_price(symbol=arguments['symbol'])
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": output
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process the messages, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [6]:
helper.generate_response(client,stock_assistant,"Can you please provide me the latest stock prices for Microsoft and NVIDIA?", "123", "Alex", function_calling_delegate)
helper.generate_response(client,stock_assistant,"Can you please provide me the latest stock prices for Apple and Tesla?", "123", "Alex", function_calling_delegate)


Creating new thread for Alex with user_id 123
Added thread:  thread_9nSMq2tnIRvTq2MxeKbnfWQE 1
Function Calling
{'tool_calls': [{'id': 'call_UORd3x2mMWH0JkBn9Lt6FcSn', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_s0xs1PIzYZREvTec3x5mgklO', 'function': {'arguments': '{"symbol": "NVDA"}', 'name': 'get_stock_price'}, 'type': 'function'}]}
Submitting outputs back to the Assistant...
User Alex: Can you please provide me the latest stock prices for Microsoft and NVIDIA?
Assistant:
The latest stock prices are as follows:

- Microsoft (MSFT): $388.76
- NVIDIA (NVDA): $555.55


Retrieving existing thread for Alex with user_id 123
Function Calling
{'tool_calls': [{'id': 'call_yUrd9UATfbdAKVCODfCHsm7t', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_sge3c5btsglLlkq9GuXZQaS7', 'function': {'arguments': '{"symbol": "TSLA"}', 'name': 'get_stock_price'}, 'type': 'function'}]

('Can you please provide me the latest stock prices for Apple and Tesla?',
 'The latest stock prices are as follows:\n\n- Apple (AAPL): $182.05\n- Tesla (TSLA): $213.81')

### Delete all demo objects

In [7]:
helper.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_AJ5ghb5I9q7ZCVnk9WQxxuBJ', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_9nSMq2tnIRvTq2MxeKbnfWQE', deleted=True, object='thread.deleted')
Deleting:  0  files.
